# info
- by: LeeHaEun
- start: 21.11.22 Mon
- end: 21.11.22 Mon
- review: 21.11.22 Mon

# theme:Saving and loading models

- 훈련된 모델을 저장 가능 -> 효율
- 텐서플로우 등의 배포 라이브러리 중 하나를 사용
  - 텐서플로우 라이트: 안드로이드, ios 등에서 가능
  - 텐서플로우 js: 자바스크립트 환경에서 모델 배포하기 위함

# code_l07c01_saving_and_loading_models

## import

In [ ]:
!pip install -U tensorflow_hub
!pip install -U tensorflow_datasets

In [ ]:
import time
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tensorflow.keras import layers

## part1: Load the Cats vs. Dogs Dataset

In [ ]:
(train_examples, validation_examples), info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)

In [ ]:
def format_image(image, label):
  # `hub` image modules exepct their data normalized to the [0,1] range.
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return  image, label

num_examples = info.splits['train'].num_examples

BATCH_SIZE = 32
IMAGE_RES = 224

train_batches      = train_examples.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)

## Part 2: Transfer Learning with TensorFlow Hub

In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES,3))

In [ ]:
# 훈련 못시키게 동결시켜버리기
feature_extractor.trainable = False

In [ ]:
# 트랜스퍼 러닝으로 불러온 레이어로 레이어 세팅
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(2)
])

model.summary()

In [ ]:
# 모델 훈련시키기
model.compile(
  optimizer='adam', 
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

EPOCHS = 3
history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

In [ ]:
# 클래스 이름 설정
class_names = np.array(info.features['label'].names)
class_names

In [ ]:
image_batch, label_batch = next(iter(train_batches.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()
predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]
predicted_class_names

In [ ]:
# 실제로 확인해봅시다
print("Labels: ", label_batch)
print("Predicted labels: ", predicted_ids)

plt.figure(figsize=(10,9))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  color = "blue" if predicted_ids[n] == label_batch[n] else "red"
  plt.title(predicted_class_names[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")

## Part 3: Save as Keras `.h5` model

In [ ]:
t = time.time()

export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

In [ ]:
!ls

## Part 4:  Load the Keras `.h5` Model

In [ ]:
reloaded = tf.keras.models.load_model(
  export_path_keras, 
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.summary()

In [ ]:
result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

In [ ]:
(abs(result_batch - reloaded_result_batch)).max()
# = 0 ; 두 모델의 예측이 같다

##keep training

In [ ]:
# reload & 계속해서 트레이닝!
EPOCHS = 3
history = reloaded.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

## Part 5: Export as SavedModel

In [ ]:
t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)

tf.saved_model.save(model, export_path_sm) #모델, 경로 -> 저장

In [ ]:
!ls {export_path_sm}

## Part 6: Load SavedModel

In [ ]:
# tf_saved_model.load(): 세이브한 모델을 불러온다
reloaded_sm = tf.saved_model.load(export_path_sm)
# reloaded_sm: 세이브한 모델로 예측 돌리기
reload_sm_result_batch = reloaded_sm(image_batch, training=False).numpy()
# 이전의 모델과 세이브한 모델의 차이 확인
(abs(result_batch - reload_sm_result_batch)).max() # = 0

## Part 7: Loading the SavedModel as a Keras Model


In [ ]:
t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)
tf.saved_model.save(model, export_path_sm)

In [ ]:
# tf_saved_model.load() = 케라스 레이어x(fit, predict, summary 없음)
# => tf.keras.models.load_model 사용 필요
reload_sm_keras = tf.keras.models.load_model(
  export_path_sm,
  custom_objects={'KerasLayer': hub.KerasLayer})

reload_sm_keras.summary()

In [ ]:
result_batch = model.predict(image_batch)
reload_sm_keras_result_batch = reload_sm_keras.predict(image_batch)

In [ ]:
(abs(result_batch - reload_sm_keras_result_batch)).max()

## Part 8:  Download your model

In [ ]:
# local disk에서 zio파일로 다운로드 가능
!zip -r model.zip {export_path_sm}
!ls

In [ ]:
# file.download(): 현재 작업 디렉토리에서 파일 검색
try:
  from google.colab import files
  files.download('./model.zip')
except ImportError:
  pass